In [2]:
import os
import sys
import numpy as np
import pandas as pd
import scipy
import math
import random
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.decomposition import TruncatedSVD
from surprise import Reader, Dataset, SVD, evaluate, dump, accuracy
from collections import defaultdict

# Custom libraries
sys.path.append('../Util')
from loader import get_books, get_book_dataframe, get_book_features
from joiner import get_ratings, get_joint, load_amazon, load_goodreads
from reduction import reduce_matrix, get_sparse

In [48]:
def get_predictions(user_to_concept, V, user_bias, global_bias):
    pred_ratings = np.zeros(len(V))
    for i in range(len(V)):
        pred = global_bias + new_user_bias + np.dot(new_user_P, qi[i])
        pred_ratings[i] = pred
    return pred_ratings

In [49]:
def get_top_n_recs(result, books, n, q):
    recs = []
    for i in range(len(result)):
        if q[i] == 0: # book user hasn't already rated
            recs.append((i, result[i]))
        else:
            recs.append((i, float('-inf'))) 
            # recs.append((i, result[i])) #leave this to verify things actually working
    recs = sorted(recs, key=lambda tup: tup[1], reverse=True)

    top_titles = []
    for i in range(n):
        book_id = recs[i][0]
        title = books.iloc[book_id]['title']
        top_titles.append(title)
    return top_titles

In [5]:
# Set this to where you save and load all data
data_path = '../../goodbooks-10k/'

In [6]:
# Get dataframe from books
books = get_book_dataframe(data_path)

found books_dataframe in file...


In [7]:
filename = '../.tmp/svd_100_300.npy'
qi = np.load(filename)

In [8]:
qi.shape

(10000, 300)

In [101]:
# user from goodreads
# sparse_q = scipy.sparse.load_npz('../.tmp/cached_users/user_likes_mystery_scifi_hates_fantasy.npz')
# sparse_q = scipy.sparse.load_npz('../.tmp/cached_users/user_likes_fantasy.npz')
sparse_q = scipy.sparse.load_npz('../.tmp/cached_users/user_nickgreenquist.npz')
q = sparse_q.toarray()
q = np.array(q[0].tolist())
new_user = np.copy(q)
new_user

array([3, 3, 0, ..., 0, 0, 0])

In [102]:
# undo the rating mapping we usually do

# Turn 1-5 rating scale into negative - positive scale
# original mapper: ratings_mapper = {0:0, 1:-2, 2:-1, 3:1, 4:2, 5:3}
ratings_mapper = {0:0, -2:-1, -1:-2, 1:3, 2:4, 3:5}
for i in range(len(q)):
    new_user[i] = ratings_mapper[new_user[i]]
new_user

array([5, 5, 0, ..., 0, 0, 0])

In [103]:
# create array of indices of books this user has actually rated
indices = []
for i in range(len(new_user)):
    if new_user[i] != 0:
        indices.append(i)

In [158]:
# Hyperparams
learning_rate = 0.01
user_bias_reg = 0.1
P_reg = 0.001
global_bias = 3.946136

# 25 updates per rated book
iterations = len(indices) * 25

 # see total loss 10 times
show_total_loss = iterations / 10

n_factors = qi.shape[1]
cols = qi.shape[0]

# TODO: save item biases after training with Surprise
item_bias = np.full(10000, 4.0);

In [159]:
# 1. get the user_bias for this user
new_user_bias = 0

In [160]:
# 2. set up new random P
mu, sigma = 0, 0.1
new_user_P = np.random.normal(mu, (sigma / n_factors), n_factors)

In [162]:
# 3. computer small number of iterations of SGD
for iteration in range(iterations):
    
    # TODO: is there a way to not redo ALL errors after just one item update??
    
    # 3.1 calculate loss with current components
    errors = np.zeros(shape=(cols), dtype=np.float)
    for i in indices:
        rating = new_user[i]
        # pred = global_bias + new_user_bias + item_bias[i] + np.dot(new_user_P, qi[i])
        pred = new_user_bias + np.dot(new_user_P, qi[i])
        errors[i] = rating - pred
    

    # 3.2 periodically calculate total loss and output
    if iteration % show_total_loss == 0:
        total_loss = 0.0
        for j in indices:
            total_loss += pow(errors[j], 2)
        print("Loss at Iteration {}: {}".format(iteration, total_loss))

    # 3.3 run single SGD iteration
    i = random.choice(indices)

    # update P
    for f in range(n_factors):
        p_update = learning_rate * (errors[i] * qi[i][f] - P_reg * new_user_P[f])
        new_user_P[f] += p_update

    # update user bias
    ub_update = learning_rate * (errors[i] - user_bias_reg * new_user_bias)
    new_user_bias += ub_update


Loss at Iteration 0: 249.57045934045692
Loss at Iteration 505: 215.11055207583487
Loss at Iteration 1010: 223.4407353010497
Loss at Iteration 1515: 200.37341456146817
Loss at Iteration 2020: 183.26329867905932
Loss at Iteration 2525: 184.96313958213838
Loss at Iteration 3030: 191.17734120180992
Loss at Iteration 3535: 173.8658084082772
Loss at Iteration 4040: 178.84108342962801
Loss at Iteration 4545: 167.36259446980685


In [163]:
recs = get_top_n_recs(get_predictions(new_user_P, qi, new_user_bias, global_bias), books, 25, new_user)
for r in recs:
    print(r)

Bad Monkey
Herzog
The Hundred Thousand Kingdoms (Inheritance Trilogy, #1)
The Drunkard's Walk: How Randomness Rules Our Lives
Nudge: Improving Decisions About Health, Wealth, and Happiness
ظل الأفعى
Here's to Us
If You Could See Me Now
Breaking Night: A Memoir of Forgiveness, Survival, and My Journey from Homeless to Harvard
How the Light Gets In (Chief Inspector Armand Gamache, #9)
The Next Best Thing
Blood Memory
Black-Eyed Susans
The Work of Art in the Age of Its Technological Reproducibility, and Other Writings on Media
The Story of Babar
Girl at War
The Gift of the Magi
Hogwarts: An Incomplete and Unreliable Guide (Pottermore Presents, #3)
Millennium Snow, Vol. 1
Hamilton: The Revolution
Unveiled: Tamar (Lineage of Grace #1)
Wool (Wool, #1)
The Handmaid's Tale
Caraval
Acheron (Dark-Hunter #14)
